# Should My Child Eat This?

### One of the biggest challenges I’ve experienced as a mother is getting my son to eat healthy food. He prefers processed food such as crackers or cookies. I'm always hesitant to given him what he wants in fear of the food being unhealthy. For that reason, I've decided to build a model that can determine if a food is healthy or not. 

In [25]:
# Imports
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
import math

from sklearn import datasets
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from sklearn.linear_model import LinearRegression
import math
from sklearn.linear_model import Lasso

%matplotlib inline
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [2]:
data = pd.read_csv('en.openfoodfacts.org.products.tsv', sep='\t', low_memory=False)

In [3]:
data.head()

,code,url,creator,created_t,created_datetime,last_modified_t,last_modified_datetime,product_name,generic_name,quantity,...,fruits-vegetables-nuts_100g,fruits-vegetables-nuts-estimate_100g,collagen-meat-protein-ratio_100g,cocoa_100g,chlorophyl_100g,carbon-footprint_100g,nutrition-score-fr_100g,nutrition-score-uk_100g,glycemic-index_100g,water-hardness_100g
0,0000000003087,http://world-en.openfoodfacts.org/product/0000...,openfoodfacts-contributors,1474103866,2016-09-17T09:17:46Z,1474103893,2016-09-17T09:18:13Z,Farine de blé noir,NaN,1kg,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,0000000004530,http://world-en.openfoodfacts.org/product/0000...,usda-ndb-import,1489069957,2017-03-09T14:32:37Z,1489069957,2017-03-09T14:32:37Z,Banana Chips Sweetened (Whole),NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,14.0,14.0,NaN,NaN
2,0000000004559,http://world-en.openfoodfacts.org/product/0000...,usda-ndb-import,1489069957,2017-03-09T14:32:37Z,1489069957,2017-03-09T14:32:37Z,Peanuts,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN
3,0000000016087,http://world-en.openfoodfacts.org/product/0000...,usda-ndb-import,1489055731,2017-03-09T10:35:31Z,1489055731,2017-03-09T10:35:31Z,Organic Salted Nut Mix,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,12.0,12.0,NaN,NaN
4,0000000016094,http://world-en.openfoodfacts.org/product/0000...,usda-ndb-import,1489055653,2017-03-09T10:34:13Z,1489055653,2017-03-09T10:34:13Z,Organic Polenta,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
data.columns

Index(['code', 'url', 'creator', 'created_t', 'created_datetime',
       'last_modified_t', 'last_modified_datetime', 'product_name',
       'generic_name', 'quantity',
       ...
       'fruits-vegetables-nuts_100g', 'fruits-vegetables-nuts-estimate_100g',
       'collagen-meat-protein-ratio_100g', 'cocoa_100g', 'chlorophyl_100g',
       'carbon-footprint_100g', 'nutrition-score-fr_100g',
       'nutrition-score-uk_100g', 'glycemic-index_100g',
       'water-hardness_100g'],
      dtype='object', length=163)

In [5]:
for column in data.columns:
    if len(data[str(column)].value_counts()) == 1:
        data.drop([str(column)],axis=1,inplace=True)

In [6]:
data.describe()

,no_nutriments,additives_n,ingredients_from_palm_oil_n,ingredients_from_palm_oil,ingredients_that_may_be_from_palm_oil_n,ingredients_that_may_be_from_palm_oil,nutrition_grade_uk,energy_100g,energy-from-fat_100g,fat_100g,...,fruits-vegetables-nuts_100g,fruits-vegetables-nuts-estimate_100g,collagen-meat-protein-ratio_100g,cocoa_100g,chlorophyl_100g,carbon-footprint_100g,nutrition-score-fr_100g,nutrition-score-uk_100g,glycemic-index_100g,water-hardness_100g
count,0.0,283867.000000,283867.000000,0.0,283867.000000,0.0,0.0,295367.000000,869.000000,2.794970e+05,...,3228.000000,404.000000,182.000000,1383.000000,0.0,278.000000,254856.000000,254856.000000,0.0,0.0
mean,NaN,1.876851,0.023430,NaN,0.059736,NaN,NaN,1125.453320,587.216617,5.606587e+04,...,33.392680,60.360124,15.362637,52.102675,NaN,335.790664,9.166137,8.980656,NaN,NaN
std,NaN,2.501022,0.153094,NaN,0.280660,NaN,NaN,936.825952,713.255708,2.963385e+07,...,32.906834,29.262350,3.692658,19.028361,NaN,423.244817,8.999870,9.151757,NaN,NaN
min,NaN,0.000000,0.000000,NaN,0.000000,NaN,NaN,0.000000,0.000000,0.000000e+00,...,0.000000,0.000000,8.000000,6.000000,NaN,0.000000,-15.000000,-15.000000,NaN,NaN
25%,NaN,0.000000,0.000000,NaN,0.000000,NaN,NaN,382.000000,49.400000,1.000000e-01,...,0.000000,45.000000,12.000000,33.000000,NaN,82.650000,1.000000,1.000000,NaN,NaN
50%,NaN,1.000000,0.000000,NaN,0.000000,NaN,NaN,1092.000000,300.000000,5.290000e+00,...,25.000000,58.000000,15.000000,52.000000,NaN,190.950000,10.000000,9.000000,NaN,NaN
75%,NaN,3.000000,0.000000,NaN,0.000000,NaN,NaN,1674.000000,900.000000,2.000000e+01,...,55.000000,93.000000,15.000000,70.000000,NaN,378.700000,16.000000,16.000000,NaN,NaN
max,NaN,30.000000,2.000000,NaN,6.000000,NaN,NaN,231199.000000,3830.000000,1.566667e+10,...,100.000000,100.000000,25.000000,100.000000,NaN,2842.000000,40.000000,40.000000,NaN,NaN


In [9]:
data.drop(['url','creator','created_t', 'created_datetime', 'last_modified_datetime', 'generic_name',
          'no_nutriments', 'ingredients_from_palm_oil','ingredients_that_may_be_from_palm_oil', 
           'nutrition_grade_uk', 'chlorophyl_100g','glycemic-index_100g','water-hardness_100g', 
           '-butyric-acid_100g', '-caproic-acid_100g','image_small_url','image_url', 'main_category', 
           'states_tags', 'states', 'additives_tags', 'traces_tags', 'last_modified_t', 'packaging_tags', 
           'brands_tags', 'categories_tags', 'categories', 'origins_tags', 'manufacturing_places_tags', 
           'labels', 'labels_tags', 'emb_codes_tags', 'countries_tags','countries','traces'], axis=1, inplace=True)

ValueError: labels ['url' 'creator' 'created_t' 'created_datetime' 'last_modified_datetime'
 'generic_name' 'no_nutriments' 'ingredients_from_palm_oil'
 'ingredients_that_may_be_from_palm_oil' 'nutrition_grade_uk'
 'chlorophyl_100g' 'glycemic-index_100g' 'water-hardness_100g'
 '-butyric-acid_100g' '-caproic-acid_100g' 'image_small_url' 'image_url'
 'main_category' 'states_tags' 'states' 'additives_tags' 'traces_tags'
 'last_modified_t' 'packaging_tags' 'brands_tags' 'categories_tags'
 'categories' 'origins_tags' 'manufacturing_places_tags' 'labels'
 'labels_tags' 'emb_codes_tags' 'countries_tags' 'countries' 'traces'] not contained in axis

In [8]:
data.describe()

,additives_n,ingredients_from_palm_oil_n,ingredients_that_may_be_from_palm_oil_n,energy_100g,energy-from-fat_100g,fat_100g,saturated-fat_100g,-capric-acid_100g,-lauric-acid_100g,-arachidic-acid_100g,...,caffeine_100g,taurine_100g,ph_100g,fruits-vegetables-nuts_100g,fruits-vegetables-nuts-estimate_100g,collagen-meat-protein-ratio_100g,cocoa_100g,carbon-footprint_100g,nutrition-score-fr_100g,nutrition-score-uk_100g
count,283867.000000,283867.000000,283867.000000,295367.000000,869.000000,2.794970e+05,263823.000000,2.000000,4.000000,27.000000,...,91.000000,33.000000,52.000000,3228.000000,404.000000,182.000000,1383.000000,278.000000,254856.000000,254856.000000
mean,1.876851,0.023430,0.059736,1125.453320,587.216617,5.606587e+04,5.092460,6.040000,36.136182,10.383852,...,2.075503,12.262055,6.476138,33.392680,60.360124,15.362637,52.102675,335.790664,9.166137,8.980656
std,2.501022,0.153094,0.280660,936.825952,713.255708,2.963385e+07,7.965148,0.226274,24.101433,3.939718,...,7.725321,69.605988,2.013490,32.906834,29.262350,3.692658,19.028361,423.244817,8.999870,9.151757
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000e+00,0.000000,5.880000,0.044730,0.064000,...,0.000000,0.000720,0.000000,0.000000,0.000000,8.000000,6.000000,0.000000,-15.000000,-15.000000
25%,0.000000,0.000000,0.000000,382.000000,49.400000,1.000000e-01,0.000000,5.960000,34.661183,7.100000,...,0.016500,0.035000,6.275000,0.000000,45.000000,12.000000,33.000000,82.650000,1.000000,1.000000
50%,1.000000,0.000000,0.000000,1092.000000,300.000000,5.290000e+00,1.790000,6.040000,47.600000,12.800000,...,0.021000,0.039000,7.200000,25.000000,58.000000,15.000000,52.000000,190.950000,10.000000,9.000000
75%,3.000000,0.000000,0.000000,1674.000000,900.000000,2.000000e+01,7.140000,6.120000,49.075000,13.200000,...,0.042500,0.400000,7.500000,55.000000,93.000000,15.000000,70.000000,378.700000,16.000000,16.000000
max,30.000000,2.000000,6.000000,231199.000000,3830.000000,1.566667e+10,550.000000,6.200000,49.300000,15.400000,...,44.000000,400.000000,8.400000,100.000000,100.000000,25.000000,100.000000,2842.000000,40.000000,40.000000


In [10]:
data.shape

(356027, 123)

In [11]:
def describe_categorical(X):
    from IPython.display import display, HTML
    display(HTML(X[X.columns[X.dtypes == "object"]].describe().to_html()))

In [12]:
describe_categorical(data)

,code,product_name,quantity,packaging,brands,categories_en,origins,manufacturing_places,labels_en,emb_codes,first_packaging_code_geo,cities,cities_tags,purchase_places,stores,countries_en,ingredients_text,allergens,allergens_en,traces_en,serving_size,additives,additives_en,ingredients_from_palm_oil_tags,ingredients_that_may_be_from_palm_oil_tags,nutrition_grade_fr,pnns_groups_1,pnns_groups_2,states_en,main_category_en
count,356001,338515,119285,89960,326977,103301,25050,42008,59204,32490,20872,26,22502,66565,57701,355752,283893,37176,20,28420,216621,283825,170189,6572,14328,254856,129006,132689,355975,103249
unique,356001,249245,15563,16466,66961,24112,5244,8076,16004,9210,1696,3,2733,5741,3646,779,238920,16575,20,3926,25928,228280,39941,14,180,5,14,42,1068,3639
top,8025488000153,Ice Cream,500 g,"Sachet,Plastique",Carrefour,"Beverages,Non-sugared beverages",France,France,"Organic,EU Organic,fr:AB Agriculture Biologique",EMB 56251E,"47.633333,-2.666667",a,theix-morbihan-france,France,Carrefour,United States,"Carbonated water, natural flavor.",blé,http://en.openfoodfacts.org/images/products/30...,Nuts,240 ml (8 fl oz),[ extra-virgin-olive-oil -> en:extra-virgin-o...,E330 - Citric acid,huile-de-palme,e471-mono-et-diglycerides-d-acides-gras-alimen...,d,unknown,unknown,"To be completed,Nutrition facts completed,Ingr...",Plant-based foods and beverages
freq,1,411,5285,2633,3670,2345,6159,11225,5080,229,297,11,286,14543,6878,173159,222,1495,1,2491,5496,397,8656,6239,3584,72436,43603,43603,167862,22950


In [13]:
data.drop(['code', 'product_name', 'quantity', 'packaging', 'brands', 'categories_en', 'origins', 
           'manufacturing_places', 'labels_en', 'emb_codes', 'first_packaging_code_geo', 'cities', 
           'cities_tags', 'purchase_places', 'stores', 'countries_en', 'ingredients_text', 
           'allergens', 'allergens_en', 'traces_en', 'serving_size', 'additives', 'additives_en', 
           'ingredients_from_palm_oil_tags', 'ingredients_that_may_be_from_palm_oil_tags', 
           'nutrition_grade_fr', 'pnns_groups_1', 'pnns_groups_2', 'states_en', 'main_category_en'], axis=1, inplace=True)


data.describe()

,additives_n,ingredients_from_palm_oil_n,ingredients_that_may_be_from_palm_oil_n,energy_100g,energy-from-fat_100g,fat_100g,saturated-fat_100g,-capric-acid_100g,-lauric-acid_100g,-arachidic-acid_100g,...,caffeine_100g,taurine_100g,ph_100g,fruits-vegetables-nuts_100g,fruits-vegetables-nuts-estimate_100g,collagen-meat-protein-ratio_100g,cocoa_100g,carbon-footprint_100g,nutrition-score-fr_100g,nutrition-score-uk_100g
count,283867.000000,283867.000000,283867.000000,295367.000000,869.000000,2.794970e+05,263823.000000,2.000000,4.000000,27.000000,...,91.000000,33.000000,52.000000,3228.000000,404.000000,182.000000,1383.000000,278.000000,254856.000000,254856.000000
mean,1.876851,0.023430,0.059736,1125.453320,587.216617,5.606587e+04,5.092460,6.040000,36.136182,10.383852,...,2.075503,12.262055,6.476138,33.392680,60.360124,15.362637,52.102675,335.790664,9.166137,8.980656
std,2.501022,0.153094,0.280660,936.825952,713.255708,2.963385e+07,7.965148,0.226274,24.101433,3.939718,...,7.725321,69.605988,2.013490,32.906834,29.262350,3.692658,19.028361,423.244817,8.999870,9.151757
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000e+00,0.000000,5.880000,0.044730,0.064000,...,0.000000,0.000720,0.000000,0.000000,0.000000,8.000000,6.000000,0.000000,-15.000000,-15.000000
25%,0.000000,0.000000,0.000000,382.000000,49.400000,1.000000e-01,0.000000,5.960000,34.661183,7.100000,...,0.016500,0.035000,6.275000,0.000000,45.000000,12.000000,33.000000,82.650000,1.000000,1.000000
50%,1.000000,0.000000,0.000000,1092.000000,300.000000,5.290000e+00,1.790000,6.040000,47.600000,12.800000,...,0.021000,0.039000,7.200000,25.000000,58.000000,15.000000,52.000000,190.950000,10.000000,9.000000
75%,3.000000,0.000000,0.000000,1674.000000,900.000000,2.000000e+01,7.140000,6.120000,49.075000,13.200000,...,0.042500,0.400000,7.500000,55.000000,93.000000,15.000000,70.000000,378.700000,16.000000,16.000000
max,30.000000,2.000000,6.000000,231199.000000,3830.000000,1.566667e+10,550.000000,6.200000,49.300000,15.400000,...,44.000000,400.000000,8.400000,100.000000,100.000000,25.000000,100.000000,2842.000000,40.000000,40.000000


In [14]:
#Impute all variables with mean
for v in data:
    data[str(v)].fillna(data[str(v)].mean(), inplace=True)

In [15]:
data.describe()

,additives_n,ingredients_from_palm_oil_n,ingredients_that_may_be_from_palm_oil_n,energy_100g,energy-from-fat_100g,fat_100g,saturated-fat_100g,-capric-acid_100g,-lauric-acid_100g,-arachidic-acid_100g,...,caffeine_100g,taurine_100g,ph_100g,fruits-vegetables-nuts_100g,fruits-vegetables-nuts-estimate_100g,collagen-meat-protein-ratio_100g,cocoa_100g,carbon-footprint_100g,nutrition-score-fr_100g,nutrition-score-uk_100g
count,356027.000000,356027.000000,356027.000000,356027.000000,356027.000000,3.560270e+05,356027.000000,356027.000000,356027.000000,356027.000000,...,356027.000000,356027.000000,356027.000000,356027.000000,356027.000000,356027.000000,356027.000000,356027.000000,356027.000000,356027.000000
mean,1.876851,0.023430,0.059736,1125.453320,587.216617,5.606587e+04,5.092460,6.040000,36.136183,10.383852,...,2.075503,12.262055,6.476138,33.392680,60.360124,15.362637,52.102675,335.790664,9.166137,8.980656
std,2.233229,0.136702,0.250608,853.293243,35.217965,2.625639e+07,6.856595,0.000379,0.069962,0.033668,...,0.122828,0.659904,0.024099,3.132887,0.984512,0.083260,1.185535,11.805671,7.614510,7.743016
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000e+00,0.000000,5.880000,0.044730,0.064000,...,0.000000,0.000720,0.000000,0.000000,0.000000,8.000000,6.000000,0.000000,-15.000000,-15.000000
25%,0.000000,0.000000,0.000000,494.000000,587.216617,9.000000e-01,0.300000,6.040000,36.136182,10.383852,...,2.075503,12.262055,6.476138,33.392680,60.360124,15.362637,52.102675,335.790664,3.000000,3.000000
50%,1.876851,0.000000,0.000000,1125.453320,587.216617,1.224000e+01,5.000000,6.040000,36.136182,10.383852,...,2.075503,12.262055,6.476138,33.392680,60.360124,15.362637,52.102675,335.790664,9.166137,8.980656
75%,2.000000,0.000000,0.000000,1569.000000,587.216617,4.750000e+01,5.092460,6.040000,36.136182,10.383852,...,2.075503,12.262055,6.476138,33.392680,60.360124,15.362637,52.102675,335.790664,14.000000,14.000000
max,30.000000,2.000000,6.000000,231199.000000,3830.000000,1.566667e+10,550.000000,6.200000,49.300000,15.400000,...,44.000000,400.000000,8.400000,100.000000,100.000000,25.000000,100.000000,2842.000000,40.000000,40.000000


In [16]:
# data.astype(numpy.float64)

In [17]:
for column in data.columns:
    if data[column].isnull().any():
            print (column)

-lignoceric-acid_100g
-cerotic-acid_100g
-melissic-acid_100g
-elaidic-acid_100g
-mead-acid_100g
-erucic-acid_100g
-nervonic-acid_100g


In [18]:
data.drop(['-lignoceric-acid_100g', '-cerotic-acid_100g', '-melissic-acid_100g', '-elaidic-acid_100g', 
           '-mead-acid_100g', '-erucic-acid_100g', '-nervonic-acid_100g'], axis=1, inplace=True)

In [19]:
y1=data.pop('nutrition-score-fr_100g')
y2=data.pop('nutrition-score-uk_100g')

In [ ]:
# want to compute the average of our ys
y = []

In [20]:
X_train, X_test, y_train, y_test = train_test_split(data, y2, test_size=.2, random_state=42)

In [21]:
model = RandomForestRegressor(n_estimators=100, oob_score=True, n_jobs=-1, random_state=42)
model.fit(X_train, y_train)

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_split=1e-07, min_samples_leaf=1,
           min_samples_split=2, min_weight_fraction_leaf=0.0,
           n_estimators=100, n_jobs=-1, oob_score=True, random_state=42,
           verbose=0, warm_start=False)

In [23]:
list(zip (y_test, model.predict(X_test)))

[(8.980655742850864, 8.9737441030328498),
 (20.0, 20.0),
 (18.0, 17.23),
 (8.980655742850864, 8.9737441030328498),
 (12.0, 11.879613114857017),
 (8.980655742850864, 8.9806557428510896),
 (15.0, 15.0),
 (20.0, 19.969999999999999),
 (8.980655742850864, 8.9806557428507237),
 (8.980655742850864, 8.9806557428507876),
 (11.0, 11.01),
 (18.0, 18.23),
 (-3.0, -2.8999999999999999),
 (23.0, 23.0),
 (8.980655742850864, 8.9737441030328498),
 (5.0, 14.58),
 (9.0, 9.0),
 (-1.0, -1.0),
 (0.0, 0.0),
 (16.0, 15.6),
 (18.0, 18.0),
 (19.0, 18.940000000000001),
 (2.0, 2.0),
 (-2.0, -2.04),
 (8.980655742850864, 8.9806557428510896),
 (23.0, 23.0),
 (21.0, 20.890000000000001),
 (0.0, 0.0),
 (9.0, 8.9399999999999995),
 (10.0, 10.449999999999999),
 (12.0, 12.0),
 (-3.0, -2.2799999999999998),
 (-6.0, -6.5499999999999998),
 (8.980655742850864, 8.9806557428506952),
 (14.0, 14.0),
 (9.0, 9.0600000000000005),
 (0.0, 0.37),
 (-6.0, -6.0),
 (6.0, 5.8200000000000003),
 (25.0, 24.47980655742851),
 (1.0, 1.0),
 (8.98065

In [26]:
y_hat = model.predict(X_test)
mse = mean_squared_error(y_test, y_hat)
print(mse)

0.804936223866


In [27]:
rmse = math.sqrt(mse)
print(rmse)

0.8971823804925603


In [28]:
r2 = r2_score(y_test, y_hat)
print(r2)

0.986608508123
